In [ ]:
import os
import openai

from getpass import getpass
from langchain.llms import OpenAI
from pyspark.sql import SparkSession
from langchain.agents import create_spark_dataframe_agent

In [ ]:
os.environ['OPENAI_API_KEY'] = openai.api_key = getpass("Insert your key:")

In [ ]:
spark = SparkSession.builder.getOrCreate()
csv_file_path = "data/hr_ibm.csv"
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)
df.show(3)

In [ ]:
agent = create_spark_dataframe_agent(llm=OpenAI(temperature=0), df=df, verbose=True)

In [ ]:
agent.run("how many rows are there?")